In [1]:
import pandas as pd
from sqlite3 import connect
from datetime import datetime as dt
from IPython.display import Image,display
from openpyxl import load_workbook


In [2]:
conn = connect('history.db')

###  Check by Model


In [ ]:
# EP-6000
models = ['VP-7000',
         'BL-7000',
         'EG-760R',
         'EG-760Z',
         'EG-740N',
         'EC-760R-V/I',
         'EC-760ZP-V/L'
          
        ]

model = models[2]
q = f'''
        SELECT [rma no.],model,serial_no,repair_status,approval
        
        FROM consolidated 
        WHERE model = '{model}'
    '''
model_list = pd.read_sql(q,conn)
model_list.head()

In [ ]:
#part PRICES
rma_list = tuple(model_list['RMA No.'])
q = f'''
        SELECT DISTINCT p.PART_NO,p.PART_DESCRIPTION,pn.short_name,
        ROUND(pr.[Dealer Standard Price (wiout VAT)],0) AS [Dealer Price]
        FROM (parts p
        LEFT JOIN part_name pn ON p.PART_DESCRIPTION = pn.PART_DESCRIPTION)
        LEFT JOIN prices pr ON REPLACE(p.part_no,'_','') = pr.[Article Code]
        WHERE [rma no.] IN {rma_list}
        
        ORDER BY P.PART_DESCRIPTION
    '''
print(f'Model: {model}')
parts_price = pd.read_sql(q,conn)
parts_price

In [ ]:
parts_no_price

In [ ]:
# part no price
parts_no_price = ''
parts_no_price = parts_price[parts_price['Dealer Price'].isnull()]
parts_no_price = parts_no_price.reset_index()
if len(parts_no_price) > 0:
    # parts_list
    parts_list =''
    if len(parts_no_price)==1:
        parts_list = f" = '{parts_no_price['PART_NO'][0][:-1]}'"
    else:
        parts_list = f"IN {tuple (parts_no_price['PART_NO'].str.slice(stop=-1))}"
    
    # Replacement
    q = f'''
            SELECT [Article Code] AS PART_NO,[Parts Name(EN)] AS PART_DESCRIPTION,
            ROUND([Dealer Standard Price (wiout VAT)],0) AS [Dealer Price]

            FROM prices pr

            WHERE pr.family {parts_list}
        '''
    replacements = pd.read_sql(q,conn)
    display(replacements)
else:
    print('Done')
    replacements=''
    parts_list=''

In [ ]:
file_name ='Service Contract/ND1'
parts_price.to_excel(f'{file_name}-{model}-parts.xlsx',sheet_name = f'{model}-parts',index = False)
try:
    replacements.to_excel(f'{file_name}-{model}-replace.xlsx', sheet_name = f'{model}-replace',index=False)
except Exception as e:
    print(e)

In [ ]:
replacements

In [ ]:
replacement

In [ ]:
import xlwings as xw
file_name = f'Service Contract/ND1_240117.xlsx'
wb = xw.Book()
wb.save(file_name)
wb.close()

In [ ]:
replacements

In [ ]:


book = load_workbook(file_name)
writer = pd.ExcelWriter(file_name, engine = 'openpyxl')
writer.book = book

parts_price.to_excel(writer, sheet_name = f'parts1-{model}',index=False)
try:
    replacements.to_excel(writer, sheet_name = f'parts-{model}',index=False)
except Exception as e:
    print(e)
writer.close()


In [ ]:
parts_no_price['PART_NO']

In [ ]:
pd.read_sql('select * from prices',conn)